<img width="200px" align="left" style="margin-right:20px" src="https://static.tildacdn.com/tild3038-6565-4361-a462-366365643138/new_logo.png"> <br /><br /><br /><br /><b>New Professions Lab</b> <br /> Специалист по большим данным

# Проект: Спрогнозировать пол и возрастную категорию интернет-пользователей по логу посещения сайтов

Одна из задач DMP-системы состоит в том, чтобы по разрозненным даннным, таким, как посещения неким пользователем сайтов, классифицировать его и присвоить ему определённую категорию: пол, возраст, интересы и так далее. В дальнейшем составляется портрет, или профиль, пользователя, на основе которого ему более таргетированно показывается реклама в интернете.

### Задача

Используя доступный набор данных о посещении страниц у одной части пользователей, сделать прогноз относительно **пола и возрастной категории** другой части пользователей. Угадывание (hit) - правильное предсказание и пола, и возрастной категории одновременно.

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os, sys
import json
import pickle
import glob
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize
import pymorphy2
from tqdm.auto import tqdm
from nltk.corpus import words, stopwords
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

### Обработка данных на вход

Файл содержит данные о посещении сайтов ~40 000 пользователей, при этом по некоторым из них (~ 35 000) известны их пол и возрастная категория, а по 5 000 - эта информация не известна. В файле есть 4 поля:
* **gender** - пол, принимающий значения `M` (male - мужчина), `F` (female - женщина), `-` (пол неизвестен);
* **age** - возраст, представленный в виде диапазона x-y (строковый тип), или `-` (возрастная категория неизвестна);
* **uid** - идентификатор пользователя, строковая переменная;
* **user_json** - поле json, в котором содержатся записи о посещении сайтов этим пользователем `(url, timestamp)`.

In [8]:
df = pd.read_csv('gender_age_dataset.txt', sep='\t')

In [9]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


In [10]:
df.iloc[0].user_json

'{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}'

Видим, что это некая сериализованная json-строка, которую можно легко разобрать через модуль `json`:

In [11]:
# df = df.loc[df['gender'] != '-']
df['user_json'] = df['user_json'].apply(lambda x:" ".join(pd.json_normalize(json.loads(x)['visits']).url))
df['user_json'] = df['user_json'].apply(lambda x: re.sub("[^а-яА-яa-zA-z]", " ", x).lower().split())
df['user_json'] = df['user_json'].apply(lambda x:" ".join(x))

In [12]:
df['user_json'].head(5)

0    http zebra zoya ru chehol organayzer dlja macb...
1    http sweetrading ru p http sweetrading ru p ht...
2    http ru oriflame com products product code htt...
3    http translate tattoo ru font selection hash c...
4    https mail rambler ru folder http news rambler...
Name: user_json, dtype: object

In [15]:
def words_only(text):
    text = BeautifulSoup(text).get_text()      
    words = re.sub("[^а-яА-яa-zA-z]", " ", text).lower().split()
    stopw = set(['http', 'ru', 'com', 'net', 'www', 'url', 'html', 'search', 'tv', 'su', 'net', 
                 'https', 'user','index', 'login'])                  
    cleaned_words = [w for w in words if not w in stopw]  
    cleaned_words = [w for w in cleaned_words if (len(w) > 3) & (len(w) < 20)]
    return(" ".join(cleaned_words))

In [16]:
df['user_json'] = df['user_json'].apply(lambda x: words_only(x))

In [18]:
texts_count = df.shape[0]
clean_texts = []

for i in range(0, texts_count):
    clean_texts.append(words_only(df['user_json'][i]))

clean_texts[0]

'zebra zoya chehol organayzer dlja macbook grid utm_campaign utm_content utm_medium utm_source begun news yandex yandsearch chezasite delay story sotovik news zaderzhivaetsja news yandex yandsearch chezasite delay story sotovik news zaderzhivaetsja'

In [19]:
df['target'] = df['gender'] + ':' + (df['age'])

df.head()

,gender,age,uid,user_json,target
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,zebra zoya chehol organayzer dlja macbook grid...,F:18-24
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,sweetrading sweetrading sweetrading port_chann...,M:25-34
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,oriflame products product code oriflame produc...,F:25-34
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,translate tattoo font selection hash nadietah ...,F:25-34
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,mail rambler folder news rambler mail rambler ...,M:>=55


In [20]:
df.target.value_counts()

M:25-34    8666
F:25-34    6791
M:35-44    5089
-:-        5000
F:35-44    4271
F:18-24    2886
F:45-54    2597
M:45-54    2147
M:18-24    2012
F:>=55      895
M:>=55      784
Name: target, dtype: int64

In [22]:
%%time 

tf_idf = TfidfVectorizer(max_features=20000)

m_tf_idf = tf_idf.fit_transform(clean_texts)

Wall time: 26.5 s


In [26]:
m_tf_idf.toarray()[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
# Сохранение векторайзера

vectorizer = 'project01_vectorizer.pickle'

with open(os.environ['HOME'] + '/' + vectorizer, 'wb') as f:
    pickle.dump(tf_idf, f)

### Построение модели

In [24]:
%%time 

xgb = XGBClassifier(n_estimators=300, max_depth=10, learning_rate=0.1, random_state=42)

xgb.fit(m_tf_idf, df['target'])

Wall time: 1h 12min 3s


XGBClassifier(max_depth=10, n_estimators=300, objective='multi:softprob',
              random_state=42)

In [25]:
# Сохранение модели

model_file = 'project01_model.pickle'

with open(os.environ['HOME'] + '/' + model_file, 'wb') as f:x
    pickle.dump(xgb, f)

### Подбор параметров

In [27]:
# X_train, X_test, y_train, y_test = train_test_split(m_tf_idf, df['target'], test_size=0.2, random_state=42)

# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# estimator = XGBClassifier(class_weight='balanced', random_state=42)

# parameters = {'max_depth': np.arange(5, 10),
#               'n_estimators': np.arange(100, 501, 50),
#               'learning_rate': [0.1, 0.5]}

# xbg_grid_cv = GridSearchCV(estimator, parameters, scoring='accuracy', n_jobs=-1, cv=cv)
# xbg_grid_cv.fit(X_train, y_train)

# xbg_grid_cv.best_params_

### Пайплайн модели на тестовых данных

In [28]:
columns=['gender','age','uid','user_json']

test_df = pd.read_table(sys.stdin, sep='\t', header=None, names=columns)

test_df['user_json'] = test_df['user_json'].apply(lambda x:" ".join(json_normalize(json.loads(x)['visits']).url))
test_df['user_json'] = test_df['user_json'].apply(lambda x: re.sub("[^a-zA-z]", " ", x).lower().split())
test_df['user_json'] = test_df['user_json'].apply(lambda x:" ".join(x))

test_df['user_json'] = test_df['user_json'].apply(lambda x: words_only(x))

model_file = 'project01_model.pickle'
p1 = pickle.load(open(model_file, 'rb'))

vectorizer = 'project01_vectorizer.pickle'
tf_idf = pickle.load(open(vectorizer, 'rb'))

m_tf_idf = tf_idf.transform(test_df['user_json'])

test_df['target'] = p1.predict(m_tf_idf)
test_df['gender'] = test_df['target'].str[:1]
test_df['age'] = test_df['target'].str[2:]

output = test_df[['uid', 'gender', 'age']]
output.sort_values(by='uid',axis = 0, ascending = True, inplace = True)
sys.stdout.write(output.to_json(orient='records'))